In [1]:
import cv2
import numpy as np
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, Input
import time

In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
path = './weights/only_weight/'
model = enhance_net(input_shape=(None,None,3), train=False)
# model.enhancement_net.summary()
model.enhancement_net.load_weights(path)

In [4]:
def show_img(img_array):
    print(np.shape(img_array))
    b,h,w,c = np.shape(img_array)
    plt.figure(figsize=(int(h/2), int(w/2)))
    for index_h, i in enumerate(img_array):
        plt.subplot(2, int(b/2), index_h+1)
        plt.imshow(i)
    plt.show()

In [5]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [7]:
dir_path = './image_analysis/high/'
for i in range(1,1075):
    # ------------------讀取影像--------------------------
    img_path = './image_analysis/low/{0}.JPG'.format(i)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
    h,w,c = np.shape(img)
    # -----------------------end--------------------------


    # ------------轉換為輸入模型的圖像形狀------------------
    pred_img = tf.image.resize(img, [512,512])
    pred_img = tf.reshape(pred_img,(1,512,512,c))
    # -----------------------end--------------------------


    # -----------------------預測-------------------------
    parameter_map = model.predict(pred_img)
    # -----------------------end--------------------------


    # 將參數圖 放大到與 讀取影像一樣大小
    parameter_map = tf.reshape(parameter_map,(512,512,c))
    parameter_map = tf.image.resize(parameter_map, [h,w])
    parameter_map = tf.reshape(parameter_map,(1,h,w,c))
    pred_img = tf.cast(tf.reshape(img, (1,h,w,c)),dtype=tf.float32)
    # -----------------------end--------------------------


    # ----------------對low-light Image增強----------------
    enhancement_images = []
    for j in range(8):
        pred_img = pred_img + parameter_map * (pred_img - tf.math.pow(pred_img, 2))
        pred_img = NormalizeData(pred_img)
        enhancement_image = np.reshape(pred_img, (h,w,c))*255
        if j == 6 :
            cv2.imwrite(dir_path + "{0}.jpg".format(i), cv2.cvtColor(np.asarray(enhancement_image), cv2.COLOR_RGB2BGR))
        enhancement_images.append(enhancement_image)
    # -----------------------end--------------------------

# Image Fusion

In [ ]:

def readImagesAndTimes(dir_index):
  
  filenames = [
               "./image/{0}/0.jpg".format(dir_index),
               "./image/{0}/1.jpg".format(dir_index),
               "./image/{0}/2.jpg".format(dir_index),
               "./image/{0}/3.jpg".format(dir_index),
               "./image/{0}/4.jpg".format(dir_index),
               ]
 
  images = []
  for filename in filenames:
    # im = cv2.imdecode(np.fromfile(filename, dtype=np.uint8), cv2.IMREAD_COLOR)
    im = cv2.imread(filename)
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    images.append(im)
  
  return images

In [ ]:
if __name__ == '__main__':
  
  # Read images
  print("Reading images ... ")
  
  # Read example images
  images = readImagesAndTimes(dir_index)
  needsAlignment = False
  
  # Align input images
  if needsAlignment:
    print("Aligning images ... ")
    alignMTB = cv2.createAlignMTB()
    alignMTB.process(images, images)
  else :
    print("Skipping alignment ... ")
    
  
  # Merge using Exposure Fusion
  print("Merging using Exposure Fusion ... ")
  mergeMertens = cv2.createMergeMertens()
  exposureFusion = mergeMertens.process(images)
  # exposureFusion = NormalizeData(exposureFusion)
  plt.imshow(exposureFusion)
  plt.show()
 
  # Save output image
  print("Saving output ... exposure-fusion.jpg")
  exposureFusion = cv2.cvtColor(exposureFusion,cv2.COLOR_RGB2BGR)
  cv2.imwrite("./image/{0}/exposure-fusion.jpg".format(dir_index), exposureFusion*255)